In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

dataset = tf.keras.utils.get_file(
    fname="10_food_classes_10_percent.zip", 
    origin="https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip", 
    extract=True,
)

current_directory = os.path.join(os.path.dirname(dataset)) + "\\10_food_classes_10_percent"

In [2]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = current_directory + "\\train\\"
test_dir = current_directory + "\\test\\"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir, 
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")

print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

def create_model(model_url, num_classes=10):
    feature_extractor_layer = hub.KerasLayer(model_url,
     trainable=False,
     name="feature_extractor_layer",
     input_shape=IMAGE_SHAPE+(3,))
    model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
    ])
    return model


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


In [3]:
resnet_model = create_model(
    resnet_url,
    num_classes=train_data_10_percent.num_classes
)

resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [4]:
resnet_history = resnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data,
    validation_steps=len(test_data),
)

Epoch 1/5
24/24 [==============================] - 28s 847ms/step - loss: 1.9591 - accuracy: 0.3467 - val_loss: 1.1965 - val_accuracy: 0.6276
Epoch 2/5
24/24 [==============================] - 17s 734ms/step - loss: 0.9161 - accuracy: 0.7147 - val_loss: 0.8550 - val_accuracy: 0.7248
Epoch 3/5
24/24 [==============================] - 17s 737ms/step - loss: 0.6293 - accuracy: 0.8240 - val_loss: 0.7453 - val_accuracy: 0.7592
Epoch 4/5
24/24 [==============================] - 17s 730ms/step - loss: 0.4761 - accuracy: 0.8827 - val_loss: 0.6990 - val_accuracy: 0.7732
Epoch 5/5
24/24 [==============================] - 17s 725ms/step - loss: 0.3823 - accuracy: 0.9080 - val_loss: 0.6849 - val_accuracy: 0.7712
